In [12]:
from typing import Union, List
from langchain_ollama import OllamaEmbeddings
from langchain_elasticsearch import ElasticsearchStore, DenseVectorStrategy

def load_elastic_vectorstore(index_names: Union[str, List[str]]):
    # 단일 문자열인 경우 리스트로 변환
    if isinstance(index_names, str):
        index_names = [index_names]
    
    vector_store = ElasticsearchStore(
        index_name=index_names, 
        embedding=OllamaEmbeddings(
            base_url="http://localhost:11434", 
            model="bge-m3:latest"
        ), 
        es_url="http://localhost:9200",
        es_user="Kstyle",
        es_password="12345",
        # strategy=DenseVectorStrategy(hybrid=True)
    )
    return vector_store


index_names = ["ship_safety"]
vector_store = load_elastic_vectorstore(index_names=index_names)
vector_store

In [13]:
results = vector_store.similarity_search(
    query="중대재해의 정의",
    k=5,
    filter=[{"term": {"metadata.lv2_cat.keyword": "가이드"}}],
)
results

[Document(metadata={'filename': '안전보건 실무길잡이_선박건조 및 수리업.pdf', 'page': '0', 'lv1_cat': '안전', 'lv2_cat': '가이드'}, page_content='This page explains 안전보건 실무길잡이_선박건조 및 수리업 that belongs to 안전 and  가이드 categories.\n안전보건 실무길잡이\n\n선박건조 및 수리업\n\n<!-- image -->\n\n산업재해예방\n\n안전보건공단\n\n<!-- image -->')]

In [14]:
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold", 
    search_kwargs={"score_threshold": 0.6},
)
retriever.invoke("중대재해의 정의")

[Document(metadata={'filename': '안전보건 실무길잡이_선박건조 및 수리업.pdf', 'page': '32'}, page_content='This page explains 안전보건 실무길잡이_선박건조 및 수리업 that belongs to 안전 and  가이드 categories.\n<!-- image -->\n\n## Check Box\n\n## 용어의 정의\n\n- 사업주는 산업재해 조사표에 근로자 대표의 확인올 받아야 하여, 그 기재 내용에 대하여\n- 산업재해가 발생한 경우 다음 사항울 기록하고 3년간 보존한다.\n- 사업장의 개요 및 근로자의 인적사항, 재해 발생의 일시 및 장소 재해 발생의 원인 및 과정, 재해 재발방지 계획\n- 하는 경우 갈음활 수있음\n\n## 중대재해\n\n0 사망자가 1명 이상 발생한 재해   0 3개월 이상의 요양이 필요한 부상자가 동시에 2명 이상 발생한 재해 부상자 또는 직업성 질병자가 동시에 10명 이상 발생한 재해름 말한다.\n\n## 지체없이\n\n정당한 사유재해 등으로 인한 통신수단의 이용이 곤란하거나 재해자 응급구호; 2차 재해 재발방지루 위한 조치\n\n## 3일 이상의 휴업\n\n(휴업일수에 사고 발생일은 포함되지 않으나 법정공휴일 또는 근로 제공 의무가 없는 휴무일은 포함)\n\n## 작업중지 요건\n\n0 고용노동부장관은 . (중락).. 해당 사업장에 산재가 다시 발생할 급박한 위험이 있다고 판단 되는 경우 그 작업의 중지름 명할 수 있음\n\n- 중대재해가 발생한 해당 직업 중대재해가 발생한 작업과 동일한 작업\n\n0고용노동부장관은 토사 구축물의봉과 화재 폭발 유해하거나 위험한 물질의 누출 등으로 인해 중대재해가 발생해 . (중락). 확산월 수 있다고 판단되는 등 불가피한 경우 해당사업장의\n\n<!-- image -->'),
 Document(metadata={'filename': '중대재해 처벌 등에 관한 법률(법률)(제17907호)(20220127).pdf', 'page'

In [15]:
retriever = vector_store.as_retriever(
    search_type="mmr", 
    search_kwargs={"fetch_k": 10, "k":5},
)
retriever.invoke("중대재해의 정의")

[Document(metadata={'filename': '안전보건 실무길잡이_선박건조 및 수리업.pdf', 'page': '32'}, page_content='This page explains 안전보건 실무길잡이_선박건조 및 수리업 that belongs to 안전 and  가이드 categories.\n<!-- image -->\n\n## Check Box\n\n## 용어의 정의\n\n- 사업주는 산업재해 조사표에 근로자 대표의 확인올 받아야 하여, 그 기재 내용에 대하여\n- 산업재해가 발생한 경우 다음 사항울 기록하고 3년간 보존한다.\n- 사업장의 개요 및 근로자의 인적사항, 재해 발생의 일시 및 장소 재해 발생의 원인 및 과정, 재해 재발방지 계획\n- 하는 경우 갈음활 수있음\n\n## 중대재해\n\n0 사망자가 1명 이상 발생한 재해   0 3개월 이상의 요양이 필요한 부상자가 동시에 2명 이상 발생한 재해 부상자 또는 직업성 질병자가 동시에 10명 이상 발생한 재해름 말한다.\n\n## 지체없이\n\n정당한 사유재해 등으로 인한 통신수단의 이용이 곤란하거나 재해자 응급구호; 2차 재해 재발방지루 위한 조치\n\n## 3일 이상의 휴업\n\n(휴업일수에 사고 발생일은 포함되지 않으나 법정공휴일 또는 근로 제공 의무가 없는 휴무일은 포함)\n\n## 작업중지 요건\n\n0 고용노동부장관은 . (중락).. 해당 사업장에 산재가 다시 발생할 급박한 위험이 있다고 판단 되는 경우 그 작업의 중지름 명할 수 있음\n\n- 중대재해가 발생한 해당 직업 중대재해가 발생한 작업과 동일한 작업\n\n0고용노동부장관은 토사 구축물의봉과 화재 폭발 유해하거나 위험한 물질의 누출 등으로 인해 중대재해가 발생해 . (중락). 확산월 수 있다고 판단되는 등 불가피한 경우 해당사업장의\n\n<!-- image -->'),
 Document(metadata={'filename': '중대재해 처벌 등에 관한 법률(법률)(제17907호)(20220127).pdf', 'page'